In [ ]:
"""
This block gets all the popular repo data from Github
"""
import requests
import json
def get_repo_master_data(target_stars, date_since, file_name):
    json_content = {'data': []}
    for i in xrange(1,11):
        url = 'https://api.github.com/search/repositories?q=created:%3E{}+stars:%3E{}&sort=stars&order=desc&per_page=100&page={}'.format(date_since, target_stars, i)
        res = requests.get(url)
        json_content['data'] += res.json()['items']
    #saves the data to a file in json format
    with open(file_name, 'w') as f:
        f.write(json.dumps(json_content, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
"""
Test get repo data function
"""
def get_repo_master_data():
    target_stars = 5000
    date_since = '2000-11-06'
    file_name = 'repo_master_data_json.txt'
    get_repo_data(target_stars, date_since, file_name)

In [ ]:
"""
Testing block
"""
get_repo_master_data()

In [7]:
file_name = 'repo_master_data_json.txt'
with open(file_name, 'r') as f:
    data = json.load(f)
    print len(data['data'])
print json.dumps(data['data'][0], sort_keys=True, indent=4, separators=(',', ': '))

1000
{
    "archive_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/{archive_format}{/ref}",
    "assignees_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/assignees{/user}",
    "blobs_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/git/blobs{/sha}",
    "branches_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/branches{/branch}",
    "clone_url": "https://github.com/FreeCodeCamp/FreeCodeCamp.git",
    "collaborators_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/collaborators{/collaborator}",
    "comments_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/comments{/number}",
    "commits_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/commits{/sha}",
    "compare_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/compare/{base}...{head}",
    "contents_url": "https://api.github.com/repos/FreeCodeCamp/FreeCodeCamp/contents/{+path}",
    "contributors_url": "https://api.github.co

In [10]:
def get_repo_commit_data(file_name, repo_master_file_name):
    client_id = '{your_id}'
    client_secret = '{your_secret}'
    with open(repo_master_file_name, 'r') as f:
        master_data = json.load(f)
        commits_data = {}
        for repo in master_data['data']:
            # commits_url, name
            url = repo['commits_url']
            commits_url = url[:len(url)-6] + '?client_id={}&client_secret={}'.format(client_id, client_secret)
            res = requests.get(commits_url)
            commits_data[repo['name']] = res.json()
            print repo['name'], len(commits_data), res.status_code
        with open(file_name, 'w') as f:
            f.write(json.dumps(commits_data, sort_keys=True, indent=4, separators=(',', ': ')))

In [11]:
get_repo_commit_data('repo_commit_data_json.txt', 'repo_master_data_json.txt')

FreeCodeCamp 1 200
bootstrap 2 200
free-programming-books 3 200
d3 4 200
react 5 200
angular.js 6 200
awesome 7 200
Font-Awesome 8 200
You-Dont-Know-JS 9 200
oh-my-zsh 10 200
javascript 11 200
gitignore 12 200
jquery 13 200
react-native 14 200
linux 15 200
electron 16 200
tensorflow 17 200
docker 18 200
animate.css 19 200
node-v0.x-archive 20 200
meteor 21 200
html5-boilerplate 22 200
swift 23 200
vue 24 200
rails 25 200
atom 26 200
reveal.js 27 200
impress.js 28 200
nw.js 29 200
Semantic-UI 30 200
legacy-homebrew 31 200
node 32 200
three.js 33 200
the-art-of-command-line 34 200
moment 35 200
socket.io 36 200
express 37 200
jekyll 38 200
AFNetworking 39 200
laravel 40 200
httpie 41 200
material-design-icons 42 200
ionic 43 200
awesome-python 44 200
brackets 45 200
Chart.js 46 200
backbone 47 200
redux 48 200
hacker-scripts 49 200
jQuery-File-Upload 50 200
resume.github.com 51 200
material-design-lite 52 200
foundation-sites 53 200
gulp 54 200
Front-end-Developer-Interview-Questions 55 